Тестирование весов модели, полученных с использованием Metric Learning (ArcFace Loss)

In [40]:
import torch
from torchvision.transforms import transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torchvision.models import resnet18

import pytorch_lightning as pl
from pytorch_metric_learning import distances, losses, reducers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator
from pytorch_metric_learning.utils.inference import CustomKNN

import matplotlib.pyplot as plt

from PIL import Image
import json
import warnings
warnings.filterwarnings('ignore')

Словарь с метками

In [26]:
dict_path = '/Users/chervonikov_alexey/Desktop/VK Video Intern/notebooks/metric_learning/labels/idx2classname.json'

with open(dict_path, 'r') as json_file:
    idx2classname = json.load(json_file)
    
# print(idx2classname)

Дополнительные функции

In [29]:
# Базовый transform
base_transform = transforms.Compose(
    [
        transforms.Resize((300, 300)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Денормализация изображения для вывода через plt.imhsow()
def unnormalize(tensor, mean=mean, std=std):
    for t, m, s in zip(tensor, mean, std):
        t.mul_(s).add_(m)
    return tensor

def plot_image(image, label):
	'''
	Вспомогательная функция для plotting'а тензора torch
	
	Параметры:
	-image: TorchTensor
	-label: str для написания заголовка
	'''
	img = image.squeeze(0).permute(1, 2, 0).numpy()
	fig = plt.figure(figsize = (4, 4))
	plt.imshow(img)
	plt.title(label)
	plt.tight_layout()
	plt.axis('off')
    


Класс модели

In [12]:
class modelArcFaceLoss(pl.LightningModule):

	'''
	Класс модели с функцией потерь ArcFaceLoss (наследует методы из pl.LightningModule)
	'''

	def __init__(
			self,
			model=resnet18(pretrained=True), # базовая модель resnet18
			embedding_size=128,
			distance_metric=distances.CosineSimilarity(),
			reducer=reducers.ThresholdReducer(low=0),
			loss_fn=losses.ArcFaceLoss,  # Вот она ArcFace из pytorch_metric_learning
			arcface_margin=0.5,  # margin гиперпараметр
			arcface_scale=64,  #scale гиперпараметр
			optimizer=Adam, 
			optimizer_params={'lr': 0.001, 'weight_decay': 0.0001},
			class_dict=idx2classname,
			min_lr=1e-5,
			step_size=8,
			gamma=0.5
			):
		
		'''
		Конуструктор объекта класс def __init__(self, ...)

		Парамеры:
		-model: Базовая модель (default: resnet18(pretrained = True))
		-embedding_size: Размер эмбеддингов после сверточных слоев для решения задачи Metric Learning (default = 128)
		-distance_metric: Метрика подсчета расстояния между объектами (default: CosineSimilarity())
		-reduce: Функция редукции потерь, которая используется для фильтрации значений loss на основе порогового значения.
		Например, ThresholdReducer(low=0) игнорирует все значения потерь ниже 0.
		Это может повысить устойчивость к шуму в данных (default: ThresholdReducer(low=0))
		-loss_fn: функция потерь (default: ArcFaceLoss)
		-arcface_margin: Смещение угла в формуле функции потерь (default: 0.5)
		-arcface_scale: Масшатабирующий параметр в формуле функции потерь (default: 64)
		-optimizer: оптимизатор (default: Adam)
		-optimizer_params: параметры оптимизатора
		-class_dict: словарь Dict label->idx2classname
		-min_lr: минимальный шаг сходимости (тот предел, до которого уменьшается lr в процессе обучения)
		-step_size: число эпох, через которое экпоненциально уменьшаем шаг сходимости
		-gamma: уменьшающий множитель 

		Инициализирует всё необходимое

		'''

		super(modelArcFaceLoss, self).__init__()

		# Модель и её параметры (Архитектура + Функция потерь + Оптимизатор)
		self.backbone = model,
		self.backbone = self.backbone[0]
		self.embedding_size = embedding_size
		self.backbone.fc = nn.Linear(self.backbone.fc.in_features, self.embedding_size)
		self.fc = nn.Linear(self.embedding_size, self.embedding_size)
		self.distance = distance_metric
		self.reducer = reducer
		self.arcface_margin = arcface_margin
		self.arcface_scale = arcface_scale
		self.loss_fn = loss_fn(
			num_classes=len(class_dict),
			embedding_size=self.embedding_size,
			margin=self.arcface_margin,
			scale=self.arcface_scale
		)

		self.optimizer_params = optimizer_params
		self.optimizer = optimizer(self.parameters(), **self.optimizer_params)
		self.class_dict = class_dict

		# Если мы хотим еще параллельно решать задачу классификации на основе привычной CrossEntropy
		self.classifier_head = nn.Sequential(
			nn.ReLU(),
			nn.Linear(in_features=self.embedding_size, out_features=len(self.class_dict))
		)
		self.classif_loss = torch.nn.CrossEntropyLoss()
		self.save_hyperparameters()
		self.gamma = gamma
		self.step_size = step_size
		self.scheduler = StepLR(self.optimizer, step_size=self.step_size, gamma=self.gamma)
		self.min_lr = min_lr

		# Эмбеддинги для подсчета метрик в конце валидации
		self.val_embeddings = []
		self.val_labels = []

	def forward(self, input_x):
		'''
		forward модели после подачи batch_size:

		Параметры:
		-self
		-input_x: входой пакет картинок

		Возвращает эмбеддинг картинки
		'''

		# Прогон через CNN
		cnn_output = self.backbone(input_x)
		# Прогон через линейные слои
		embedding = self.fc(cnn_output)
		return embedding

	def training_step(self, batch, batch_idx):
			'''
			Часть train логики: подаем батч, разбиваем на (images, labels)
			Возвращем loss, по которому будет считаться градиент
			'''

			images, labels = batch
			embeddings = self(images)

			# ArcFace loss
			loss_arcface = self.loss_fn(embeddings, labels)
			final_loss = loss_arcface

			self.log('train_loss', final_loss, sync_dist=True)
			return final_loss

	def on_train_start(self):
		self.train()

	def validation_step(self, batch, batch_idx):
			
			'''
			Логика на валидации: подаем батч, считаем loss на валидации и записываем в tensor_board
			И добавляем эмбеддинги и метки для подсчёта метрик
			'''

			images, labels = batch
			embeddings = self(images)

			loss_arcface = self.loss_fn(embeddings, labels)

			final_loss = loss_arcface
			self.log('validation_loss', final_loss, sync_dist=True)

			self.val_embeddings.append(embeddings)
			self.val_labels.append(labels)

	def on_validation_epoch_end(self):
			
			'''
			Логика в конце валидации: считает ключевую метрики на валидации, а именно precision@1:
            
			-precision@1
			-Обнуляет массивы эмбеддингов и меток в конце
			'''

			all_embeddings = torch.cat(self.val_embeddings)
			all_labels = torch.cat(self.val_labels)

			accuracy_calculator = AccuracyCalculator(include=("precision_at_1",), k=1, knn_func=CustomKNN(
				distances.CosineSimilarity(), batch_size=64))

			metrics = accuracy_calculator.get_accuracy(all_embeddings, all_labels)
			precision_at_1 = metrics["precision_at_1"]
			self.log('precision_at_1_epoch', precision_at_1, sync_dist=True)

			self.val_embeddings = []
			self.val_labels = []

	def on_validation_start(self):
			self.eval()

	def configure_optimizers(self):
		'''
		Объявление оптимизатора и его фичей
		'''
		
		return {
			'optimizer': self.optimizer,
			'lr_scheduler': {
				'scheduler': self.scheduler,
				'interval': 'epoch',
				'frequency': 1,
				'reduce_on_plateau': False,
				'monitor': 'validation_loss',
			}
		}

	def lr_scheduler_step(self, scheduler, metric):
		'''
		Обновление шага сходимости
		'''

		scheduler.step()
		self._adjust_learning_rate()

	def _adjust_learning_rate(self):
		'''
		Проверка достижения предела learning_rate (self.min_lr)
		'''
		
		for param_group in self.optimizer.param_groups:
			param_group['lr'] = max(param_group['lr'], self.min_lr)
		
		

Необходимо научиться получать эмбеддинги для входных изображений

In [44]:
first_asics_path = '/Users/chervonikov_alexey/Desktop/VK Video Intern/notebooks/figures/asics1.jpg'
second_asics_path = '/Users/chervonikov_alexey/Desktop/VK Video Intern/notebooks/figures/asics2.jpg'

arcface_weights_path = '/Users/chervonikov_alexey/Desktop/VK Video Intern/notebooks/metric_learning/arcface_weights/best-precision-arcfaceloss-epoch=14-precision_at_1_epoch=0.95.ckpt'

def make_inference_model_arcface(image_path:str, 
								model_weights_path:str, 
								model):
	'''
	Функция для осуществления инференса
	
	Параметры:
	-image_path: путь к тестируемому изображению (str)
	-model_weights_path: путь к сохраненнымии весами (str)
	-model: класс модели
	'''

	image = Image.open(image_path).convert('RGB')
	image_tensor = base_transform(image)
	image_tensor = image_tensor.unsqueeze(0) 
	pl_model = model.load_from_checkpoint(model_weights_path)
	pl_model.eval()

	with torch.no_grad():
		output = pl_model(image_tensor).cpu()[0]
	
	return output

output_first = make_inference_model_arcface(image_path = first_asics_path, 
							model_weights_path = arcface_weights_path, 
							model = modelArcFaceLoss)


output_second = make_inference_model_arcface(image_path = second_asics_path, 
							model_weights_path = arcface_weights_path, 
							model = modelArcFaceLoss)

print(f"Cosine Similarity: {F.cosine_similarity(output_first.unsqueeze(0), output_second.unsqueeze(0))[0]:.2f}")


Cosine Similarity: 0.67


Два логотипа *ASICS* имеют близость 0.67

In [49]:
tommy_logo_path = '/Users/chervonikov_alexey/Desktop/VK Video Intern/notebooks/figures/tommy.jpg'
output_tommy_logo = make_inference_model_arcface(image_path = tommy_logo_path, 
							model_weights_path = arcface_weights_path, 
							model = modelArcFaceLoss)

print(f"Cosine Similarity: {F.cosine_similarity(output_first.unsqueeze(0), output_tommy_logo.unsqueeze(0))[0]:.2f}")

Cosine Similarity: -0.06


Видим, что разные метрики близости